In [394]:
import pandas as pd
import numpy as np

import datetime
import tempfile
import uuid
from datetime import timedelta
import requests
from arize.pandas.logger import Client, Schema
from arize.utils.types import Environments, ModelTypes

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [395]:
HotelSearch_train=pd.read_csv("./data/hotelsearch_training_predictions.csv", index_col=0)
HotelSearch_test=pd.read_csv("./data/hotelsearch_test_predictions.csv", index_col=0)

# HotelSearch_train["PredictedRelevancy"]=round(HotelSearch_train["PredictedRelevancy"],4)
# HotelSearch_test["PredictedRelevancy"]=round(HotelSearch_test["PredictedRelevancy"],4)

#HotelSearch_train["ActualRelevancyScore"].value_counts()
HotelSearch_train["prediction_ts"] = pd.to_datetime(HotelSearch_train["prediction_ts"],infer_datetime_format=True)
HotelSearch_test["prediction_ts"] = pd.to_datetime(HotelSearch_test["prediction_ts"],infer_datetime_format=True)

#HotelSearch_test.info()
a=HotelSearch_test
a.shape


(147730, 67)

In [396]:
HotelSearch_test["prediction_ts"].max()

Timestamp('2013-06-30 23:36:35')

In [397]:
def transpose(df):
    return pd.DataFrame(
        {"sequence": [df["ActualRelevancyScore"].tolist()] + [None] * (len(df) - 1)}, index=df.index
    )


HotelSearch_train = pd.concat([HotelSearch_train, HotelSearch_train.groupby("srch_id").apply(transpose)], axis=1)
HotelSearch_test = pd.concat([HotelSearch_test, HotelSearch_test.groupby("srch_id").apply(transpose)], axis=1)


In [398]:
#HotelSearch_train[["srch_id","site_id","position","ActualRelevancyScore","PredictedRelevancyScore","Predictedrank","Not_Relevant_prob","Yes_Relevant_prob","RelevanceLabelPrediction","sequence"]][HotelSearch_train.srch_id ==71]

In [399]:
#HotelSearch_test[["srch_id","site_id","position","ActualRelevancyScore","PredictedRelevancyScore","Predictedrank","Not_Relevant_prob","Yes_Relevant_prob","RelevanceLabelPrediction","sequence"]][HotelSearch_test.srch_id ==246]

In [400]:
HotelSearch_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147730 entries, 0 to 147729
Data columns (total 68 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   srch_id                      147730 non-null  int64         
 1   site_id                      147730 non-null  int64         
 2   visitor_location_country_id  147730 non-null  int64         
 3   visitor_hist_starrating      7770 non-null    float64       
 4   visitor_hist_adr_usd         7801 non-null    float64       
 5   prop_country_id              147730 non-null  int64         
 6   prop_id                      147730 non-null  int64         
 7   prop_starrating              147730 non-null  int64         
 8   prop_review_score            147499 non-null  float64       
 9   prop_brand_bool              147730 non-null  int64         
 10  prop_location_score1         147730 non-null  float64       
 11  prop_location_score2      

In [401]:
#HotelSearch_train.info()

In [402]:
drop_cols=['PredictedRelevancyScore', 'Predictedrank',
       'Not_Relevant_prob', 'Yes_Relevant_prob','RelevanceLabelPrediction','prediction_ts']
HotelSearch_test = HotelSearch_test.drop(columns=drop_cols,errors="ignore")
#HotelSearch_test['PredictedRelevancy'].value_counts()
HotelSearch_train = HotelSearch_train.drop(columns=drop_cols,errors="ignore")
#HotelSearch_train['PredictedRelevancy'].value_counts()
print(HotelSearch_test.shape)
print(HotelSearch_train.shape)

(147730, 62)
(92451, 62)


In [411]:
print(HotelSearch_train.loc[HotelSearch_test["day"] == 30].shape)

HotelSearch_train.info()

KeyError: 'day'

In [404]:
HotelSearch_train.columns
feature_column_names=[ 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price',  'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff',   'weekday',
       'week_of_year', 'hour', 'minute', 'time_epoch', 'early_night'
       ]
tag_column_names = ['srch_id','position','site_id','prop_id','predictionid']

In [405]:
#HotelSearch_test.head()

In [408]:

SPACE_KEY = "7ad2a2b"
API_KEY = "35950bc0ad321270af8"

arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

model_id = "HotelSearchRankingv2"
model_version = "1.0"
model_type = ModelTypes.SCORE_CATEGORICAL

if SPACE_KEY == "" or API_KEY == "":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print("Step 2 ✅: Import and Setup Arize Client Done! Now we can start using Arize!")

Step 2 ✅: Import and Setup Arize Client Done! Now we can start using Arize!


In [409]:
# Define a Schema() object for Arize to pick up data from the correct columns for logging
training_schema = Schema(
    prediction_id_column_name="predictionid",
    prediction_label_column_name="PredictedRelevancy",
    prediction_score_column_name="PredictedScore",
    actual_label_column_name="ActualRelevancy",
    actual_score_column_name="ActualRelevancyScore",
    actual_numeric_sequence_column_name="sequence",
    feature_column_names=feature_column_names,
    tag_column_names = tag_column_names,
)

# Logging Training DataFrame
training_response = arize_client.log(
    dataframe=HotelSearch_train,
    model_id=model_id,
    model_version=model_version,
    model_type=ModelTypes.SCORE_CATEGORICAL,
    environment=Environments.TRAINING,
    schema=training_schema,
)

# If successful, the server will return a status_code of 200
if training_response.status_code != 200:
    print(
        f"logging failed with response code {training_response.status_code}, {training_response.text}"
    )
else:
    print(f"✅ You have successfully logged training set to Arize")

Success! Check out your data at https://app.arize.com/organizations/QWNjb3VudE9yZ2FuaXphdGlvbjo2Nzk=/spaces/U3BhY2U6NzEx/models/modelName/HotelSearchRankingv2?selectedTab=dataIngestion
✅ You have successfully logged training set to Arize


In [410]:
# # changing dates for ease of visualization / to mimic recent produciton dataset
END_DATE = datetime.date.today().strftime("%Y-%m-%d")
START_DATE = (datetime.date.today() - timedelta(31)).strftime("%Y-%m-%d")


def setPredictionIDandTime(df, start, end):
    out_df = pd.DataFrame()
    dts = pd.date_range(start, end).to_pydatetime().tolist()
    print(dts)
    for dt in dts:
#        print(df["day"])
        day_df = df.loc[df["day"] == dt.day].copy()
        day_df["prediction_ts"] = int(dt.strftime("%s"))
        out_df = pd.concat([out_df, day_df], ignore_index=True)
        if dt == 1:
            print(out_df.shape)
#    out_df["prediction_id"] = [str(uuid.uuid4()) for _ in range(out_df.shape[0])]
    return out_df.drop(columns="day")


HotelSearch_test = setPredictionIDandTime(HotelSearch_test, START_DATE, END_DATE)

HotelSearch_test.shape

[datetime.datetime(2022, 4, 30, 0, 0), datetime.datetime(2022, 5, 1, 0, 0), datetime.datetime(2022, 5, 2, 0, 0), datetime.datetime(2022, 5, 3, 0, 0), datetime.datetime(2022, 5, 4, 0, 0), datetime.datetime(2022, 5, 5, 0, 0), datetime.datetime(2022, 5, 6, 0, 0), datetime.datetime(2022, 5, 7, 0, 0), datetime.datetime(2022, 5, 8, 0, 0), datetime.datetime(2022, 5, 9, 0, 0), datetime.datetime(2022, 5, 10, 0, 0), datetime.datetime(2022, 5, 11, 0, 0), datetime.datetime(2022, 5, 12, 0, 0), datetime.datetime(2022, 5, 13, 0, 0), datetime.datetime(2022, 5, 14, 0, 0), datetime.datetime(2022, 5, 15, 0, 0), datetime.datetime(2022, 5, 16, 0, 0), datetime.datetime(2022, 5, 17, 0, 0), datetime.datetime(2022, 5, 18, 0, 0), datetime.datetime(2022, 5, 19, 0, 0), datetime.datetime(2022, 5, 20, 0, 0), datetime.datetime(2022, 5, 21, 0, 0), datetime.datetime(2022, 5, 22, 0, 0), datetime.datetime(2022, 5, 23, 0, 0), datetime.datetime(2022, 5, 24, 0, 0), datetime.datetime(2022, 5, 25, 0, 0), datetime.datetime(20

(151850, 62)

In [412]:
production_schema = Schema(
    prediction_id_column_name="predictionid",
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="PredictedRelevancy",
    prediction_score_column_name="PredictedScore",
    actual_label_column_name="ActualRelevancy",
    actual_score_column_name="ActualRelevancyScore",
    actual_numeric_sequence_column_name="sequence",
    feature_column_names=feature_column_names,
#    shap_values_column_names=dict(zip(feature_column_names, shap_column_names)),
    tag_column_names = tag_column_names,
)

production_response = arize_client.log(
    dataframe=HotelSearch_test,
    model_id=model_id,
    model_version=model_version,
    model_type=ModelTypes.SCORE_CATEGORICAL,
    environment=Environments.PRODUCTION,
    schema=production_schema,
)

if production_response.status_code != 200:
    print(
        f"logging failed with response code {production_response.status_code}, {production_response.text}"
    )
else:
    print(f"✅ You have successfully logged production set to Arize")

Success! Check out your data at https://app.arize.com/organizations/QWNjb3VudE9yZ2FuaXphdGlvbjo2Nzk=/spaces/U3BhY2U6NzEx/models/modelName/HotelSearchRankingv2?selectedTab=dataIngestion
✅ You have successfully logged production set to Arize


In [360]:
HotelSearch_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154703 entries, 0 to 154702
Data columns (total 64 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   srch_id                      154703 non-null  int64  
 1   site_id                      154703 non-null  int64  
 2   visitor_location_country_id  154703 non-null  int64  
 3   visitor_hist_starrating      8245 non-null    float64
 4   visitor_hist_adr_usd         8276 non-null    float64
 5   prop_country_id              154703 non-null  int64  
 6   prop_id                      154703 non-null  int64  
 7   prop_starrating              154703 non-null  int64  
 8   prop_review_score            154470 non-null  float64
 9   prop_brand_bool              154703 non-null  int64  
 10  prop_location_score1         154703 non-null  float64
 11  prop_location_score2         122095 non-null  float64
 12  prop_log_historical_price    154703 non-null  float64
 13 